In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score
####
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
my_pipeline = make_pipeline(LogisticRegression())

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)
italian_ingredients = list(set(italian_df.ingredients))

mexican_df = DataFrame(clean_mexican_cuisine_data)
mexican_ingredients = list(set(mexican_df.ingredients))


In [5]:
#fixing data for CountVectorizer (italian_df)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)
    

In [6]:
#fixing data for CountVectorizer (mexican_df)

mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    


In [7]:
#using CountVectorizer  (italian, mexican)

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()

it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')
print(it_df)
mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()

mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')
print(mex_df)

       abbamele  accent  acinipepe  acornsquash  activedryyeast  adobosauce  \
4             0       0          0            0               0           0   
14            0       0          0            0               0           0   
20            0       0          0            0               0           0   
56            0       0          0            0               0           0   
62            0       0          0            0               0           0   
...         ...     ...        ...          ...             ...         ...   
49679         0       0          0            0               0           0   
49681         0       0          0            0               0           0   
49689         0       0          0            0               0           0   
49697         0       0          0            0               0           0   
49708         0       0          0            0               0           0   

       adoboseasoning  agavenectar  agedbalsamicvin

In [8]:
#putting mexican and italian in one DataFrame

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')
print(df['Cuisine'])

4        0
14       0
20       0
56       0
62       0
        ..
49678    1
49687    1
49707    1
49709    1
49717    1
Name: Cuisine, Length: 14276, dtype: int64


In [9]:
#not needed
#making a dictionary to keep recipe ids with the cuisine that they belong

dict_ = {}
count = 0

for id_ in df.iterrows():
    
    if count < 7838:
        
        dict_[id_[0]] = 'italian'
    
    
    else:

        dict_[id_[0]] = 'mexican'
        
    count += 1   
                
#print(dict_)

In [10]:
X = df.values  #all columns with values AFAIRESE TO CUISINE
print(X)

Y = df['Cuisine'] #just the ids in a list
#print(Y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
#kf = KFold(n_splits = 5)
#kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

#for train_index, test_index in kf.split(X):
    
    #print('TRAIN:', train_index, 'TEST:', test_index)
    #X_train, X_test = X[train_index], X[test_index]
    #Y_train, Y_test = Y[train_index], Y[test_index]

In [12]:
#scores = cross_val_score(my_pipeline, X, Y, scoring='neg_mean_absolute_error')
#print(scores)

In [13]:
#print('Mean Absolute Error %2f' %(-1 * scores.mean()))

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [15]:
#print(Y_test.shape)
#print(len(X_test))

In [16]:
# fit a model
lm = linear_model.LinearRegression()
model = lm.fit(X_train, Y_train)
predictions = lm.predict(X_test)

In [17]:
predictions[:]

array([-3.51151400e-15,  1.00000000e+00,  1.00000000e+00, ...,
        1.00000000e+00,  4.92661467e-16,  1.00000000e+00])